In [ ]:
from serpapi import GoogleSearch

def get_query(query):
    params = {
        "q": query,
        "engine": "google",
        "api_key": "d5283d8a6c6640c36e5228ae57e8baa9170859f8b5fa73e3c941cdb51afa9e0f"
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    organic_results = results['organic_results']
    return organic_results

organic_results = get_query("professor Lawrence Angrave")




In [ ]:
import trafilatura as tr
import pandas as pd

def parse_results(link):
    downloaded = tr.fetch_url(link)
    text = tr.extract(downloaded)
    if text == None:
        return ''
    return text

def get_scholar_results(author_id):
    params = {
        'engine': 'google_scholar_author',
        'author_id': author_id,
        'api_key' : 'd5283d8a6c6640c36e5228ae57e8baa9170859f8b5fa73e3c941cdb51afa9e0f'
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    return results

dataset = {'titles': [], 'text': []}

for result in organic_results:
    if 'scholar.google.com' in result['link']:
        # get user id from link
        user_id = result['link'].split('user=')[1].split('&')[0]
        # get scholar results
        scholar_results = get_scholar_results(user_id)
        print(scholar_results["author"])
    else:
        with open("Documents/" + result['title'] + '.txt', 'w') as f:
            f.write(parse_results(result['link']))
        dataset['titles'].append(result['title'])
        dataset['text'].append(parse_results(result['link']))

df = pd.DataFrame(dataset)
df.to_csv('Dataset.csv', index=False, sep='\t')

        


In [ ]:
text = ''
with open('Documents/Lawrence Angrave Professor of Computer Science.txt', 'r') as f:
    text = f.read()
print(text)

In [ ]:

from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base",max_length=2048)

In [ ]:
input_text = "Answer based on context \n\n context: v1 \n\n question: v2" 
input_text = input_text.replace('v1', text)
input_text = input_text.replace('v2', 'who is professor lawrence angrave?')
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

In [ ]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
import torch
from datasets import load_from_disk

dataset = load_from_disk('knowledge_dir/my_knowledge_dataset')
dataset.load_faiss_index(index_name= "embeddings" , file = 'knowledge_dir/my_knowledge_dataset_hnsw_index.faiss')

retriever = RagRetriever.from_pretrained('facebook/rag-sequence-nq', indexed_dataset=dataset)

# initialize with RagRetriever to do everything in one forward call
model = RagSequenceForGeneration.from_pretrained("facebook/rag-sequence-nq", retriever=retriever)
tokenizer = RagTokenizer.from_pretrained("facebook/rag-sequence-nq")

In [ ]:
inputs = tokenizer.question_encoder("Where did professor study from", return_tensors="pt")
input_ids = inputs['input_ids']
generated = model.generate(input_ids, max_length=200, num_beams=4)
generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]
print(generated_string)